<a href="https://colab.research.google.com/github/brahmbhatt-me/drone_racing_project/blob/main/Drone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚁 **Autonomous Drone Racing with Trajectory Optimization**

**Author:** Meet B  
**Program:** Master's in Robotics, Northeastern University  
**GitHub:** https://github.com/brahmbhatt-me ,
**LinkedIn:** https://www.linkedin.com/in/meet-m-brahmbhatt

---

## 📋 **Project Overview**

Welcome to this comprehensive drone racing simulation project! We'll build a complete autonomous drone system that can navigate through aerial hoops while avoiding obstacles, using state-of-the-art path planning and trajectory optimization techniques.

---

## 🎯 **What We're Building**

A drone racing simulation where the drone must fly through aerial hoops while avoiding obstacles. The complete pipeline includes:

| Step | Module | Description |
|:----:|--------|-------------|
| 1 | **Path Planning** | RRT/RRT* algorithms to find collision-free paths |
| 2 | **Drone Dynamics** | Realistic physics with thrust, drag, gravity |
| 3 | **Trajectory Optimization** | Smooth paths into efficient trajectories |
| 4 | **3D Visualization** | Interactive plotting of results |

---

## 🌍 **Real-World Applications**

This same technology powers:

| Industry | Application |
|----------|-------------|
| 📦 Logistics | Amazon/DJI delivery drones |
| 🔍 Emergency | Search-and-rescue operations |
| 🏎️ Sports | Professional drone racing (DRL, MultiGP) |
| 🎬 Film | Autonomous cinematography |
| 🚀 Aerospace | SpaceX rocket landing trajectories |

---

## 🧠 **Key Concepts You'll Learn**

| Concept | Description |
|---------|-------------|
| **6-DOF State** | Position (x, y, z) + Attitude (yaw, pitch, roll) |
| **Terminal Velocity Model** | Simplified dynamics suitable for planning |
| **RRT / RRT*** | Sampling-based path planning algorithms |
| **Direct Transcription** | Converting optimal control to nonlinear programming |
| **Constrained Optimization** | Satisfying physics while minimizing cost |

---

## 🔄 **Project Pipeline**
```
[Start] → [RRT Planning] → [Trajectory Optimization] → [3D Visualization] → [Done]
```

---

## 📁 **Project Structure**
```
drone_racing/
├── dynamics/          # Drone physics model
│   ├── drone_model.py
│   └── kinematics.py
├── planning/          # Path planning algorithms
│   ├── rrt.py
│   └── collision.py
├── optimization/      # Trajectory optimization
│   ├── optimizer.py
│   ├── cost_functions.py
│   └── constraints.py
└── visualization/     # 3D plotting
    └── plotter.py
```

---

## ▶️ **Let's Begin!**

Proceed to **Step 1** to install the required dependencies.

#  Setup and Dependencies

This section prepares the Colab environment for running the full drone racing pipeline.  
We will install the required libraries, set up numerical tools, and import helper modules used for visualization and obstacle rendering.

The setup includes:

- `numpy==1.25` for stable math operations  
- `pandas` for logging and dataset handling  
- `plotly` for 3D visualization  
- `gtsam==4.2` for factor-graph optimization  
- `helpers_obstacles.py`, a custom file for drone dynamics and plotting

Once these are installed, we can proceed to path planning with RRT/RRT* and trajectory optimization.


In [2]:
## installation

## imports
from typing import List, Tuple, Dict
import math

# Install specific versions
%pip install numpy==1.25
%pip install gtsam==4.2

import numpy as np
import pandas as pd
import unittest
import plotly.express as px
import plotly.graph_objects as go
import gtsam

rng = np.random.default_rng(12345)

# Download the helpers file from your GitHub
!wget -q -O helpers_obstacles.py https://raw.githubusercontent.com/brahmbhatt-me/drone_racing_project/main/helpers_obstacles.py

# Import helpers
import helpers_obstacles as helpers
from helpers_obstacles import Drone, axes, axes_figure

## The Problem

We're building a path planner for a racing drone that needs to navigate through suspended hoops in 3D space. Unlike traditional 2D path planning where we work with (x, y) coordinates, drone racing requires us to think in three dimensions.

The RRT algorithm is well-suited for this because it efficiently explores high-dimensional spaces without requiring a discretized grid. It works by randomly sampling points and gradually building a tree of feasible paths from start to goal.

Let's first take a look at our racing environment:

In [3]:
# visualize the drone racing track with hoops
start_race = gtsam.Pose3(r=gtsam.Rot3.Yaw(math.radians(45)), t=gtsam.Point3(1, 3, 8))
helpers.drone_racing_path(helpers.get_hoops(), start_race, [])
helpers.drone_racing_path_with_obstacles(helpers.get_hoops(), start_race, [], obstacles=helpers.get_obstacles_easy())

## Building the RRT Algorithm

The RRT algorithm follows a simple but powerful loop:
1. Sample a random point in the configuration space (with some bias toward the goal)
2. Find the nearest node in our existing tree
3. Steer from that node toward the sampled point
4. Add the new node to the tree
5. Repeat until we're close enough to the goal

**Sampling:** We introduce a "goal bias" — about 20% of the time, we return the goal itself instead of a random point. This helps the tree grow toward our destination while still exploring the space.

**Distance:** The Euclidean distance (L2 norm) measures straight-line distance between two points.

**Nearest neighbor:** We vectorize the computation using numpy — computing all distances in one operation and finding the minimum.

**Steering:** We take a small step toward the sampled point (20% of the way by default). Mathematically: `new_point = parent + fraction * (target - parent)`

In [4]:
def generate_random_point(target: gtsam.Point3) -> gtsam.Point3:
    """Generates a random point in the 10x10x10 space. Returns target 20% of the time (goal bias)."""
    if rng.random() < 0.2:
        return target
    return gtsam.Point3(rng.uniform(0, 10), rng.uniform(0, 10), rng.uniform(0, 10))


def distance_euclidean(point1: gtsam.Point3, point2: gtsam.Point3) -> float:
    """Computes the Euclidean distance between two 3D points."""
    return np.linalg.norm(point2 - point1)


def find_nearest_node(rrt: List[gtsam.Point3], node: gtsam.Point3):
    """Finds the closest node in the tree to the sampled point using vectorized operations."""
    rrt_array = np.array([np.array(p) for p in rrt])
    distances = np.linalg.norm(rrt_array - np.array(node), axis=1)
    index = np.argmin(distances)
    return rrt[index], index


def steer_naive(parent: gtsam.Point3, target: gtsam.Point3, fraction=0.2):
    """Steers from parent toward target, moving a fraction of the distance."""
    return parent + fraction * (target - parent)


def run_rrt(start, target, generate_random_node, steer, distance, find_nearest_node, threshold):
    """Main RRT loop. Builds tree from start toward target, returns tree and parent indices."""
    rrt = [start]
    parents = [-1]
    max_iterations = 2000

    for _ in range(max_iterations):
        rand_node = generate_random_node(target)
        nearest, nearest_idx = find_nearest_node(rrt, rand_node)
        new_node = steer(nearest, rand_node)
        rrt.append(new_node)
        parents.append(nearest_idx)

        if distance(new_node, target) < threshold:
            break

    return rrt, parents

## Testing the Implementation

Before running on the full problem, let's verify each function works correctly.

In [5]:
class TestRRT(unittest.TestCase):
    def test_generate_random_point(self):
        for _ in range(5):
            node = generate_random_point(gtsam.Point3(4, 5, 6))
            self.assertTrue(0 <= node[0] <= 10)
            self.assertTrue(0 <= node[1] <= 10)
            self.assertTrue(0 <= node[2] <= 10)

    def test_distance_euclidean(self):
        pt1 = gtsam.Point3(2.70109492, 4.55796488, 2.93292049)
        pt2 = gtsam.Point3(4, 7, 2)
        self.assertAlmostEqual(distance_euclidean(pt1, pt2), 2.9190804346571446, 2)

    def test_find_nearest_node(self):
        points = [gtsam.Point3(1, 2, 3), gtsam.Point3(0.90320894, 3.55218386, 3.71979848),
                  gtsam.Point3(1.52256715, 4.24174709, 3.37583879), gtsam.Point3(1.56803165, 4.10257537, 2.795647),
                  gtsam.Point3(2.68087164, 3.63713802, 4.25464017)]
        query = gtsam.Point3(3.74935314, 3.2575652, 5.20840562)
        nearest, idx = find_nearest_node(points, query)
        self.assertTrue((nearest == points[4]).all())

    def test_steer_naive(self):
        pt1 = gtsam.Point3(3.80319106, 2.49123788, 2.60348781)
        pt2 = gtsam.Point3(3.81712339, 0.33173367, 0.51835128)
        expected = gtsam.Point3(3.80597753, 2.05933704, 2.1864605)
        self.assertTrue(np.allclose(expected, steer_naive(pt1, pt2), atol=1e-2))

suite = unittest.TestSuite()
for test in ['test_generate_random_point', 'test_distance_euclidean', 'test_find_nearest_node', 'test_steer_naive']:
    suite.addTest(TestRRT(test))
unittest.TextTestRunner(verbosity=2).run(suite)

test_generate_random_point (__main__.TestRRT.test_generate_random_point) ... ok
test_distance_euclidean (__main__.TestRRT.test_distance_euclidean) ... ok
test_find_nearest_node (__main__.TestRRT.test_find_nearest_node) ... ok
test_steer_naive (__main__.TestRRT.test_steer_naive) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.009s

OK


<unittest.runner.TextTestResult run=4 errors=0 failures=0>

## Running RRT and Visualizing Results

With everything working, let's run the algorithm on a simple example — navigating from point (1, 2, 3) to point (4, 7, 2). The tree shows how RRT explores the space, branching out while being pulled toward the goal by our 20% goal bias.

In [6]:
start_rrt_3d = gtsam.Point3(1, 2, 3)
target_rrt_3d = gtsam.Point3(4, 7, 2)

rrt_3d, parents_rrt_3d = run_rrt(
    start_rrt_3d, target_rrt_3d,
    generate_random_point, steer_naive,
    distance_euclidean, find_nearest_node,
    threshold=0.1
)
print(f"Tree built with {len(rrt_3d)} nodes")
helpers.visualize_tree(rrt_3d, parents_rrt_3d, start_rrt_3d, target_rrt_3d)

Tree built with 73 nodes


Now let's extract the path from start to goal by tracing back through the parent indices.

In [7]:
def get_rrt_path(rrt: List[gtsam.Point3], parents: List[int]) -> List[gtsam.Point3]:
    """Extracts the path from start to goal by backtracking through parent indices."""
    path = []
    i = len(rrt) - 1
    while i != -1:
        path.append(rrt[i])
        i = parents[i]
    path.reverse()
    return path

path_rrt_3d = get_rrt_path(rrt_3d, parents_rrt_3d)
print(f"Path length: {len(path_rrt_3d)} waypoints")
helpers.visualize_path(path_rrt_3d, start_rrt_3d, target_rrt_3d)

Path length: 19 waypoints


## Summary

We've implemented a working 3D RRT algorithm that can find paths through open space. The key components are goal-biased random sampling, vectorized nearest-neighbor search, incremental steering, and parent tracking for path reconstruction.

In the next part, we'll extend this to handle obstacles and fly through the actual racing hoops!

# Drone Dynamics

Now that we have path finding in 3D, let's see how a drone actually flies.

We use two coordinate frames: **Navigation Frame (ENU)** where x points East, y points North, z points Up, and **Body Frame (FLU)** centered at the drone with x Forward, y Left, z Up.

We represent the drone's state using `gtsam.Pose3` which combines position (`gtsam.Point3`) and attitude (`gtsam.Rot3`).

In [8]:
# example: drone at origin with body frame aligned to navigation frame
position = gtsam.Point3(0, 0, 0)
attitude = gtsam.Rot3()
pose = gtsam.Pose3(r=attitude, t=position)
print(f"Position: {position}")
print(f"Attitude:\n{attitude}")

Position: [0. 0. 0.]
Attitude:
R: [
	1, 0, 0;
	0, 1, 0;
	0, 0, 1
]



Now we implement three key dynamics functions:

**Attitude from Euler angles:** Convert yaw, pitch, roll into a rotation matrix.

**Force computation:** Rotors provide upward thrust in body frame. When tilted, we rotate this into navigation frame: $F^n = R^n_b [0, 0, thrust]^T$

**Terminal velocity:** Drag balances thrust at max speed. Using Skydio-2 drag coefficient ($k_d = 0.0425$): $v_{terminal} = \sqrt{F / k_d}$

In [9]:
def compute_attitude_from_ypr(yaw: float, pitch: float, roll: float) -> gtsam.Rot3:
    """Computes attitude rotation matrix from yaw, pitch, roll angles (in radians)."""
    return gtsam.Rot3.Ypr(yaw, pitch, roll)


def compute_force(attitude: gtsam.Rot3, thrust: float) -> gtsam.Point3:
    """Computes force vector in navigation frame given attitude and body-frame thrust."""
    thrust_body = gtsam.Point3(0, 0, thrust)
    return attitude.rotate(thrust_body)


def compute_terminal_velocity(force: gtsam.Point3, kd: float = 0.0425) -> gtsam.Point3:
    """Computes terminal velocity from force vector and drag coefficient."""
    eps = 1e-6
    vx = np.sign(force[0]) * np.sqrt((np.abs(force[0]) + eps) / kd)
    vy = np.sign(force[1]) * np.sqrt((np.abs(force[1]) + eps) / kd)
    vz = np.sign(force[2]) * np.sqrt((np.abs(force[2]) + eps) / kd)
    return gtsam.Point3(vx, vy, vz)

Let's verify our dynamics functions pass the test cases.

In [10]:
class TestDroneDynamics(unittest.TestCase):
    def test_compute_attitude_from_ypr(self):
        yaw, pitch, roll = math.radians(45), math.radians(30), math.radians(60)
        expected = gtsam.Rot3([0.612372, 0.612372, -0.5], [-0.0473672, 0.65974, 0.75], [0.789149, -0.435596, 0.433013])
        self.assertTrue(compute_attitude_from_ypr(yaw, pitch, roll).equals(expected, tol=1e-2))

    def test_compute_force(self):
        attitude = gtsam.Rot3([0.612372, 0.612372, -0.5], [-0.0473672, 0.65974, 0.75], [0.789149, -0.435596, 0.433013])
        self.assertTrue(np.allclose(compute_force(attitude, 20.0), gtsam.Point3(15.78, -8.71, 8.66), atol=1e-2))

    def test_compute_terminal_velocity(self):
        force = gtsam.Point3(15.78, -8.71, 8.66)
        self.assertTrue(np.allclose(compute_terminal_velocity(force), gtsam.Point3(19.27, -14.32, 14.27), atol=1e-2))

suite = unittest.TestSuite()
for test in ['test_compute_attitude_from_ypr', 'test_compute_force', 'test_compute_terminal_velocity']:
    suite.addTest(TestDroneDynamics(test))
unittest.TextTestRunner(verbosity=2).run(suite)

test_compute_attitude_from_ypr (__main__.TestDroneDynamics.test_compute_attitude_from_ypr) ... ok
test_compute_force (__main__.TestDroneDynamics.test_compute_force) ... ok
test_compute_terminal_velocity (__main__.TestDroneDynamics.test_compute_terminal_velocity) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.010s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

# Steering with Terminal Velocity

Now that we understand how thrust translates to velocity, we can use realistic drone dynamics in our RRT. Since drones have *poses* (position + orientation) instead of just positions, we need to update our functions to work with `gtsam.Pose3`. Random attitudes are sampled using yaw, pitch, roll ∈ [-60°, 60°].

In [11]:
# generate random pose with 20% goal bias, random position in [0,10]³, random attitude in [-60°, 60°]
def generate_random_pose(target: gtsam.Pose3) -> gtsam.Pose3:
    """Generates random pose or returns target with 20% probability."""
    if rng.random() < 0.2:
        return target

    position = gtsam.Point3(rng.uniform(0, 10), rng.uniform(0, 10), rng.uniform(0, 10))
    deg_to_rad = np.pi / 180
    yaw = rng.uniform(-60 * deg_to_rad, 60 * deg_to_rad)
    pitch = rng.uniform(-60 * deg_to_rad, 60 * deg_to_rad)
    roll = rng.uniform(-60 * deg_to_rad, 60 * deg_to_rad)
    attitude = compute_attitude_from_ypr(yaw, pitch, roll)
    return gtsam.Pose3(attitude, position)

For poses, finding the nearest node requires considering both position and orientation. We use `helpers.distance_between_poses` which combines translational and rotational distance.

In [12]:
def find_nearest_pose(rrt: List[gtsam.Pose3], node: gtsam.Pose3):
    """Finds nearest pose in tree using combined position and orientation distance."""
    min_distance = float('inf')
    nearest, index = None, None

    for i, pose in enumerate(rrt):
        dist = helpers.distance_between_poses(pose, node)
        if dist < min_distance:
            min_distance, nearest, index = dist, pose, i
    return nearest, index

The key function: steering with physics. We compute the direction to target, orient the drone that way using `helpers.get_new_attitude`, apply max thrust (20N), calculate terminal velocity, and move for a short duration (0.1s).b

In [13]:
def steer_with_terminal_velocity(current: gtsam.Pose3, target: gtsam.Pose3, duration: float = 0.1) -> gtsam.Pose3:
    """Steers drone toward target using physics-based terminal velocity for given duration."""
    current_pos, target_pos = current.translation(), target.translation()

    # compute direction and normalize
    direction = target_pos - current_pos
    direction_norm = np.linalg.norm(direction)
    direction_normalized = direction / direction_norm if direction_norm > 1e-6 else np.array([0, 0, 1])

    # get attitude pointing toward target, apply max thrust, compute terminal velocity
    new_attitude = helpers.get_new_attitude(current, direction_normalized)
    force = compute_force(new_attitude, 20.0)
    v_terminal = compute_terminal_velocity(force)

    # move for duration at terminal velocity
    new_position = current_pos + v_terminal * duration
    return gtsam.Pose3(new_attitude, new_position)

Let's verify all three functions pass their test cases.b

In [14]:
class TestSteeringWithTerminalVelocity(unittest.TestCase):
    def test_generate_random_pose(self):
        target_node = gtsam.Pose3(r=gtsam.Rot3.Yaw(math.radians(45)), t=gtsam.Point3(8, 5, 6))
        for _ in range(5):
            node = generate_random_pose(target_node)
            self.assertTrue(np.all(np.greater_equal(node.translation(), gtsam.Point3(0, 0, 0))))
            self.assertTrue(np.all(np.less_equal(node.translation(), gtsam.Point3(10, 10, 10))))
            self.assertTrue(np.all(np.greater_equal(node.rotation().ypr(), gtsam.Point3(math.radians(-60), math.radians(-60), math.radians(-60)))))
            self.assertTrue(np.all(np.less_equal(node.rotation().ypr(), gtsam.Point3(math.radians(60), math.radians(60), math.radians(60)))))

    def test_find_nearest_pose(self):
        rrt_tree = [
            gtsam.Pose3(gtsam.Rot3([1, 0, 0], [0, 1, 0], [0, 0, 1]), gtsam.Point3(1, 2, 3)),
            gtsam.Pose3(gtsam.Rot3([0.771517, -0.617213, 0], [0.0952381, 0.119048, -0.97619], [0.617213, 0.771517, 0.154303]), gtsam.Point3(2.70427, 3.90543, 3.85213)),
            gtsam.Pose3(gtsam.Rot3([0.601649, -0.541882, 0.302815], [-0.301782, -0.62385, -0.516772], [0.627501, 0.29376, -0.721074]), gtsam.Point3(4.42268, 5.08119, 2.01005)),
            gtsam.Pose3(gtsam.Rot3([-0.696943, 0.589581, -0.36631], [-0.664345, -0.416218, 0.594076], [0.204431, 0.679463, 0.704654]), gtsam.Point3(5.40351, 6.86933, 3.83104)),
            gtsam.Pose3(gtsam.Rot3([-0.0686996, 0.218721, -0.818805], [-0.796488, -0.297401, -0.0126152], [-0.340626, 0.900832, 0.269211]), gtsam.Point3(1.43819, 5.96437, 4.97769))]
        new_node = gtsam.Pose3(gtsam.Rot3([0.682707, 0.661423, 0.310534], [-0.626039, 0.748636, -0.218217], [-0.376811, -0.0454286, 0.925176]), gtsam.Point3(5.65333, 5.65964, 1.60624))
        nearest, idx = find_nearest_pose(rrt_tree, new_node)
        self.assertTrue(nearest.equals(rrt_tree[2], tol=1e-1))
        self.assertEqual(idx, 2)

    def test_steer_with_terminal_velocity(self):
        current = gtsam.Pose3(gtsam.Rot3.Yaw(math.radians(90)), gtsam.Point3(1, 2, 3))
        target = gtsam.Pose3(gtsam.Rot3.Pitch(math.radians(45)), gtsam.Point3(8, 5, 6))
        expected = gtsam.Pose3(gtsam.Rot3([0.37, -0.86, 0], [0.31, 0.13, -0.87], [0.86, 0.37, 0.37]), gtsam.Point3(3.00, 3.31, 4.31))
        self.assertTrue(steer_with_terminal_velocity(current, target).equals(expected, tol=1e-1))

suite = unittest.TestSuite()
for test in ['test_generate_random_pose', 'test_find_nearest_pose', 'test_steer_with_terminal_velocity']:
    suite.addTest(TestSteeringWithTerminalVelocity(test))
unittest.TextTestRunner(verbosity=2).run(suite)

test_generate_random_pose (__main__.TestSteeringWithTerminalVelocity.test_generate_random_pose) ... ok
test_find_nearest_pose (__main__.TestSteeringWithTerminalVelocity.test_find_nearest_pose) ... ok
test_steer_with_terminal_velocity (__main__.TestSteeringWithTerminalVelocity.test_steer_with_terminal_velocity) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.015s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

Now let's run RRT with our physics-based steering and visualize the drone flying through the tree.

In [15]:
# run RRT with drone dynamics
start_rrt_drone = gtsam.Pose3(gtsam.Rot3(), gtsam.Point3(1, 2, 3))
target_rrt_drone = gtsam.Pose3(gtsam.Rot3(), gtsam.Point3(5, 7, 4))

rrt_drone, parents_rrt_drone = run_rrt(
    start_rrt_drone, target_rrt_drone,
    generate_random_pose, steer_with_terminal_velocity,
    helpers.distance_between_poses, find_nearest_pose,
    threshold=1.5
)
print(f"Number of RRT Nodes: {len(rrt_drone)}")

helpers.visualize_tree(rrt_drone, parents_rrt_drone, start_rrt_drone, target_rrt_drone)

# extract and animate path
path_rrt_drone = get_rrt_path(rrt_drone, parents_rrt_drone)
print(f"Path length: {len(path_rrt_drone)} waypoints")
helpers.animate_drone_path(path_rrt_drone, start_rrt_drone, target_rrt_drone)

Number of RRT Nodes: 973


Path length: 7 waypoints


# Steering with Realistic Drone Dynamics

Our previous approach had two major limitations:
1. We ignored gravitational force affecting thrust direction and magnitude
2. We allowed arbitrary instantaneous attitude changes

To hover with $g=10\ m/s^2$, the rotors must provide 10N upward thrust to compensate for gravity. Each motor can provide 0-5N, giving us acceleration range from -10 to +10 $m/s^2$.

In [16]:
# force computation now accounts for gravity pulling downward
def compute_force_with_gravity(attitude: gtsam.Rot3, thrust: float, mass: float = 1.0) -> gtsam.Point3:
    """Computes net force by combining thrust (rotated to nav frame) with gravitational force."""
    g = 10.0
    thrust_force = compute_force(attitude, thrust)
    gravity_force = gtsam.Point3(0, 0, -mass * g)
    return thrust_force + gravity_force

Let's visualize how different configurations affect the drone's motion. Each combination of yaw, pitch, roll, and thrust produces different net forces and terminal velocities.

In [17]:
# explore different flight configurations
configurations = [
    {"yaw": 90, "pitch": 0, "roll": 0, "thrust": 10},
    {"yaw": 45, "pitch": 45, "roll": 45, "thrust": 20},
    {"yaw": 30, "pitch": 0, "roll": 45, "thrust": 15},
    {"yaw": 10, "pitch": 30, "roll": 30, "thrust": 0},
]

for i, config in enumerate(configurations, 1):
    print(f"\nConfig {i}: Yaw={config['yaw']}°, Pitch={config['pitch']}°, Roll={config['roll']}°, Thrust={config['thrust']}N")
    nRb = compute_attitude_from_ypr(math.radians(config['yaw']), math.radians(config['pitch']), math.radians(config['roll']))
    net_force = compute_force_with_gravity(nRb, config['thrust'])
    terminal_velocity = compute_terminal_velocity(net_force)
    print(f"Net force: {np.round(net_force, 2)}, Terminal velocity: {np.round(terminal_velocity, 2)}")


Config 1: Yaw=90°, Pitch=0°, Roll=0°, Thrust=10N
Net force: [0. 0. 0.], Terminal velocity: [0. 0. 0.]

Config 2: Yaw=45°, Pitch=45°, Roll=45°, Thrust=20N
Net force: [17.07 -2.93  0.  ], Terminal velocity: [20.04 -8.3   0.  ]

Config 3: Yaw=30°, Pitch=0°, Roll=45°, Thrust=15N
Net force: [ 5.3  -9.19  0.61], Terminal velocity: [ 11.17 -14.7    3.78]

Config 4: Yaw=10°, Pitch=30°, Roll=30°, Thrust=0N
Net force: [  0.   0. -10.], Terminal velocity: [  0.     0.   -15.34]


## Realistic Steering

Now we implement realistic steering with constraints: attitude changes limited to ±10° per step in yaw, pitch, roll. We search over 108 combinations (3×3×3×4) of rotation deltas and thrust values [5, 10, 15, 20]N, selecting the one that gets closest to target.

The new attitude is computed as: $R^n_{b_1} = R^n_{b_0} \cdot R^{b_0}_{b_1}$ (current attitude composed with incremental rotation in body frame).

In [18]:
def steer(current: gtsam.Pose3, target: gtsam.Pose3, duration=0.1):
    """Realistic steering: searches over limited attitude changes and thrust values to find best move toward target."""
    yaw_values, pitch_values, roll_values = [-10, 0, 10], [-10, 0, 10], [-10, 0, 10]
    thrust_values = [5, 10, 15, 20]
    deg_to_rad = np.pi / 180

    best_node, min_distance = None, float('inf')
    current_attitude, current_pos = current.rotation(), current.translation()

    # search all 108 combinations
    for dyaw in yaw_values:
        for dpitch in pitch_values:
            for droll in roll_values:
                for thrust in thrust_values:
                    # incremental rotation in body frame, compose with current
                    delta_rot = compute_attitude_from_ypr(dyaw * deg_to_rad, dpitch * deg_to_rad, droll * deg_to_rad)
                    new_attitude = current_attitude.compose(delta_rot)

                    # compute motion with gravity
                    force = compute_force_with_gravity(new_attitude, thrust)
                    v_term = compute_terminal_velocity(force)
                    new_pos = current_pos + v_term * duration

                    # check if this is our best option
                    candidate = gtsam.Pose3(new_attitude, new_pos)
                    dist = helpers.distance_between_poses(candidate, target)
                    if dist < min_distance:
                        min_distance, best_node = dist, candidate

    return best_node

Let's verify both functions pass their tests.

In [19]:
class TestRealisticSteer(unittest.TestCase):
    def test_compute_force_with_gravity(self):
        attitude = gtsam.Rot3([0.612372, 0.612372, -0.5], [-0.0473672, 0.65974, 0.75], [0.789149, -0.435596, 0.433013])
        expected = gtsam.Point3(15.78, -8.71, -1.34)
        self.assertTrue(np.allclose(compute_force_with_gravity(attitude, 20.0), expected, atol=1e-2))

    def test_steer(self):
        current = gtsam.Pose3(gtsam.Rot3.Yaw(math.radians(90)), gtsam.Point3(1, 2, 3))
        target = gtsam.Pose3(gtsam.Rot3.Pitch(math.radians(45)), gtsam.Point3(8, 5, 6))
        expected = gtsam.Pose3(gtsam.Rot3([0.17, 0.97, -0.17], [-0.96, 0.20, 0.17], [0.20, 0.14, 0.97]), gtsam.Point3(1.97, 2.81, 4.49))
        self.assertTrue(steer(current, target).equals(expected, tol=1e-2))

suite = unittest.TestSuite()
for test in ['test_compute_force_with_gravity', 'test_steer']:
    suite.addTest(TestRealisticSteer(test))
unittest.TextTestRunner(verbosity=2).run(suite)

test_compute_force_with_gravity (__main__.TestRealisticSteer.test_compute_force_with_gravity) ... ok
test_steer (__main__.TestRealisticSteer.test_steer) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.011s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

Now let's run RRT with realistic drone dynamics and visualize the physically-constrained flight path.

In [20]:
# run RRT with realistic physics
start_rrt_drone_realistic = gtsam.Pose3(gtsam.Rot3(), gtsam.Point3(1, 2, 3))
target_rrt_drone_realistic = gtsam.Pose3(gtsam.Rot3(), gtsam.Point3(8, 5, 6))

rrt_drone_realistic, parents_rrt_drone_realistic = run_rrt(
    start_rrt_drone_realistic, target_rrt_drone_realistic,
    generate_random_pose, steer,
    helpers.distance_between_poses, find_nearest_pose,
    threshold=1.5
)
print(f"Nodes in RRT Tree: {len(rrt_drone_realistic)}")

helpers.visualize_tree(rrt_drone_realistic, parents_rrt_drone_realistic, start_rrt_drone_realistic, target_rrt_drone_realistic)

# extract and animate path
path_rrt_drone_realistic = get_rrt_path(rrt_drone_realistic, parents_rrt_drone_realistic)
print(f"Path length: {len(path_rrt_drone_realistic)} waypoints")
helpers.animate_drone_path(path_rrt_drone_realistic, start_rrt_drone_realistic, target_rrt_drone_realistic)

Nodes in RRT Tree: 51


Path length: 7 waypoints


# Drone Racing — Multi-Goal Path Planning

Now we put everything together for actual drone racing. The track has 4 suspended hoops that must be traversed in order. Since RRT plans point-to-point, we chain multiple RRT calls — plan to hoop 1, then from there to hoop 2, and so on.

The `pass_through_the_hoop()` helper adds final waypoints to ensure clean hoop traversal since RRT only gets us close to the goal.

In [21]:
def drone_racing_rrt(start: gtsam.Pose3, targets: List[gtsam.Pose3]) -> List[gtsam.Pose3]:
    """Plans sequential RRT paths through each hoop in order."""
    drone_path = []
    current_start = start

    for i, target in enumerate(targets):
        print(f"Planning to hoop {i+1}/{len(targets)}...")

        rrt, parents = run_rrt(
            start=current_start, target=target,
            generate_random_node=generate_random_pose, steer=steer,
            distance=helpers.distance_between_poses, find_nearest_node=find_nearest_pose,
            threshold=2.0
        )

        segment_path = get_rrt_path(rrt, parents)
        helpers.pass_through_the_hoop(target, segment_path)

        drone_path.extend(segment_path if i == 0 else segment_path[1:])
        current_start = segment_path[-1]

    return drone_path


# run the race
start_rrt_drone_race = gtsam.Pose3(r=gtsam.Rot3(), t=gtsam.Point3(1, 3, 8))
targets_rrt_drone_race = helpers.get_targets()
path_rrt_drone_race = drone_racing_rrt(start_rrt_drone_race, targets_rrt_drone_race)

helpers.drone_racing_path(helpers.get_hoops(), start_rrt_drone_race, path_rrt_drone_race)
print(f"Racing path: {len(path_rrt_drone_race)} waypoints")

Planning to hoop 1/4...
Planning to hoop 2/4...
Planning to hoop 3/4...
Planning to hoop 4/4...


Racing path: 56 waypoints


# Adding Obstacle Avoidance

Real racing environments have obstacles. We modify RRT to check collision before adding any node — if the segment from nearest node to new node intersects an obstacle, we reject it and sample again.

This produces jagged but collision-free paths, which motivates trajectory optimization later.

In [22]:
def run_rrt_with_obstacles(start, target, generate_random_node, steer, distance,
                           find_nearest_node, threshold, obstacles: List = None):
    """RRT with collision checking — rejects segments that intersect obstacles."""
    if obstacles is None:
        obstacles = []

    rrt, parents = [start], [-1]
    max_iterations = 5000

    for i in range(max_iterations):
        rand_node = generate_random_node(target)
        nearest, nearest_idx = find_nearest_node(rrt, rand_node)
        new_node = steer(nearest, rand_node)

        # extract positions for collision check
        p_nearest = nearest.translation() if isinstance(nearest, gtsam.Pose3) else nearest
        p_new = new_node.translation() if isinstance(new_node, gtsam.Pose3) else new_node

        if helpers.check_segment_collision(p_nearest, p_new, obstacles):
            continue

        rrt.append(new_node)
        parents.append(nearest_idx)

        if distance(new_node, target) < threshold:
            print(f"Goal reached in {i+1} iterations")
            break

    return rrt, parents

Testing obstacle avoidance with a sphere blocking the direct path.

In [23]:
start_obs = gtsam.Pose3(gtsam.Rot3(), gtsam.Point3(2, 2, 5))
target_obs = gtsam.Pose3(gtsam.Rot3(), gtsam.Point3(8, 8, 8))
obstacles_simple = [helpers.SphereObstacle(center=[5, 5, 6.5], radius=1.5, name="Central Pillar")]

rrt_obs, parents_obs = run_rrt_with_obstacles(
    start_obs, target_obs, generate_random_pose, steer,
    helpers.distance_between_poses, find_nearest_pose, threshold=1.5, obstacles=obstacles_simple
)

path_obs = get_rrt_path(rrt_obs, parents_obs)
has_collision, _ = helpers.check_path_collision(path_obs, obstacles_simple)
print(f"Path: {len(path_obs)} waypoints | {'Collision-free' if not has_collision else 'Has collisions'}")

helpers.visualize_path_with_obstacles(path_obs, start_obs, target_obs, obstacles_simple)

Goal reached in 26 iterations
Path: 13 waypoints | Collision-free


## Full Racing with Obstacles

Now we combine multi-goal planning with obstacle avoidance — racing through all hoops while navigating around obstacles.

In [24]:
def drone_racing_rrt_with_obstacles(start: gtsam.Pose3, targets: List[gtsam.Pose3],
                                    obstacles: List = None) -> List[gtsam.Pose3]:
    """Sequential RRT through hoops with collision avoidance."""
    if obstacles is None:
        obstacles = []

    drone_path = []
    current_start = start

    for i, target in enumerate(targets):
        print(f"Planning to hoop {i+1}/{len(targets)}...")

        rrt, parents = run_rrt_with_obstacles(
            start=current_start, target=target,
            generate_random_node=generate_random_pose, steer=steer,
            distance=helpers.distance_between_poses, find_nearest_node=find_nearest_pose,
            threshold=2.0, obstacles=obstacles
        )

        segment_path = get_rrt_path(rrt, parents)
        helpers.pass_through_the_hoop(target, segment_path)

        drone_path.extend(segment_path if i == 0 else segment_path[1:])
        current_start = segment_path[-1]

    return drone_path


# race with obstacles
start_racing = gtsam.Pose3(r=gtsam.Rot3(), t=gtsam.Point3(1, 3, 8))
targets_racing = helpers.get_targets()
obstacles_racing = helpers.get_obstacles_easy()

print(f"Racing through {len(targets_racing)} hoops with {len(obstacles_racing)} obstacles")
path_racing = drone_racing_rrt_with_obstacles(start_racing, targets_racing, obstacles_racing)

helpers.drone_racing_path_with_obstacles(
    hoops=helpers.get_hoops(), start=start_racing, path=path_racing, obstacles=obstacles_racing
)

has_collision, _ = helpers.check_path_collision(path_racing, obstacles_racing)
print(f"Total path: {len(path_racing)} waypoints | {'Collision-free' if not has_collision else 'Has collisions'}")

Racing through 4 hoops with 2 obstacles
Planning to hoop 1/4...
Goal reached in 13 iterations
Planning to hoop 2/4...
Goal reached in 26 iterations
Planning to hoop 3/4...
Goal reached in 31 iterations
Planning to hoop 4/4...
Goal reached in 416 iterations


Total path: 49 waypoints | Collision-free


# Trajectory Optimization via Direct Transcription

RRT produces collision-free but jerky paths. We'll smooth them using **direct transcription**, which converts trajectory optimization into a nonlinear programming (NLP) problem.

**State space:** x ∈ ℝ⁶ = [px, py, pz, ψ, θ, φ] (position + yaw/pitch/roll)

**Control space:** u ∈ ℝ⁴ = [Δψ, Δθ, Δφ, T] (angular changes + thrust)

**Decision variables:** z ∈ ℝ^(10N+6) — all states and controls packed into one vector

The optimizer minimizes a cost function while satisfying dynamics, boundary conditions, and control limits.

In [25]:
# helper functions for packing/unpacking decision variables and computing angle differences

def angle_diff(angle_to: float, angle_from: float) -> float:
    """Compute shortest angular difference wrapped to [-π, π]."""
    raw = angle_to - angle_from
    diff = (raw + np.pi) % (2*np.pi) - np.pi
    if raw > np.pi:
        return abs(diff)
    if raw < -np.pi:
        return -abs(diff)
    return diff


def pack_decision_vars(states: np.ndarray, controls: np.ndarray, N: int) -> np.ndarray:
    """Pack states (N+1 x 6) and controls (N x 4) into flat vector for optimizer."""
    z = np.zeros(10 * N + 6)
    for k in range(N + 1):
        z[6*k : 6*(k+1)] = states[k, :]
    offset = 6 * (N + 1)
    for k in range(N):
        z[offset + 4*k : offset + 4*(k+1)] = controls[k, :]
    return z


def unpack_decision_vars(z: np.ndarray, N: int) -> Tuple[np.ndarray, np.ndarray]:
    """Unpack flat vector back into states (N+1 x 6) and controls (N x 4)."""
    states = np.zeros((N + 1, 6))
    controls = np.zeros((N, 4))
    for k in range(N + 1):
        states[k, :] = z[6*k : 6*(k+1)]
    offset = 6 * (N + 1)
    for k in range(N):
        controls[k, :] = z[offset + 4*k : offset + 4*(k+1)]
    return states, controls

## Defining the Cost Function

The optimizer minimizes: $J(z) = J_{thrust} + J_{angular} + J_{smooth} + J_{gimbal}$

Each term captures a different aspect of good flight:
- **Thrust deviation:** Penalizes deviation from hover (T=10N) for energy efficiency
- **Angular velocity:** Penalizes rapid rotations that stress motors
- **Smoothness:** Penalizes sudden control changes (jerk minimization)
- **Gimbal lock:** Penalizes pitch near ±90° where Euler angles become singular

In [26]:
def cost_function_thrust(states: np.ndarray, controls: np.ndarray, N: int, weight: float = 0.1) -> float:
    """Penalizes thrust deviation from hover (T_hover = 10N)."""
    T_hover = 10.0
    cost = 0.0
    for k in range(N):
        thrust = controls[k, 3]
        cost += weight * (thrust - T_hover)**2
    return cost


def cost_function_angular(states: np.ndarray, controls: np.ndarray, N: int, weight: float = 1.0) -> float:
    """Penalizes large angular velocity changes."""
    cost = 0.0
    for k in range(N):
        dyaw, dpitch, droll = controls[k, 0], controls[k, 1], controls[k, 2]
        cost += weight * (dyaw**2 + dpitch**2 + droll**2)
    return cost


def cost_function_smoothness(states: np.ndarray, controls: np.ndarray, N: int, weight: float = 5.0) -> float:
    """Penalizes sudden changes in control inputs (jerk minimization)."""
    if N <= 1:
        return 0.0
    cost = 0.0
    for k in range(N - 1):
        control_diff = controls[k+1, :] - controls[k, :]
        cost += weight * np.sum(control_diff**2)
    return cost


def cost_function_gimbal_lock(states: np.ndarray, controls: np.ndarray, N: int) -> float:
    """Soft penalty when pitch approaches ±90° (gimbal lock singularity)."""
    pitch_limit = 50 * np.pi / 180
    cost = 0.0
    for k in range(N + 1):
        pitch = states[k, 4]
        if abs(pitch) > pitch_limit:
            excess = abs(pitch) - pitch_limit
            cost += 1000 * excess**2
    return cost

Now we combine all cost terms into an integrated cost function with tunable weights. Racing mode uses lower penalties for aggressive maneuvers, while normal mode prioritizes smooth, efficient flight.

In [27]:
def cost_function_tuned(z: np.ndarray, N: int, weights: Dict[str, float]) -> float:
    """Combined cost function with tunable weights for thrust, angular, and smoothness terms."""
    states, controls = unpack_decision_vars(z, N)
    cost = 0.0
    cost += cost_function_thrust(states, controls, N, weights['thrust'])
    cost += cost_function_angular(states, controls, N, weights['angular'])
    cost += cost_function_smoothness(states, controls, N, weights['smoothness'])
    cost += cost_function_gimbal_lock(states, controls, N)
    return cost

Testing all helper functions and cost functions to verify correctness.

In [29]:
class TestTrajOptHelpers(unittest.TestCase):
    def test_angle_diff_small(self):
        self.assertAlmostEqual(angle_diff(0.1, 0.0), 0.1, places=6)
        self.assertAlmostEqual(angle_diff(0.0, 0.1), -0.1, places=6)

    def test_angle_diff_wrapping(self):
        self.assertAlmostEqual(angle_diff(np.pi - 0.1, -np.pi + 0.1), 0.2, places=6)
        self.assertAlmostEqual(angle_diff(-np.pi + 0.1, np.pi - 0.1), -0.2, places=6)

    def test_pack_unpack_inverse(self):
        N = 3
        states, controls = np.random.randn(N+1, 6), np.random.randn(N, 4)
        z = pack_decision_vars(states, controls, N)
        states_rec, controls_rec = unpack_decision_vars(z, N)
        self.assertTrue(np.allclose(states, states_rec))
        self.assertTrue(np.allclose(controls, controls_rec))


class TestCostFunctions(unittest.TestCase):
    def test_cost_function_thrust_hover(self):
        controls = np.array([[0, 0, 0, 10], [0, 0, 0, 10]])
        self.assertAlmostEqual(cost_function_thrust(np.zeros((3, 6)), controls, 2, 0.1), 0.0, places=6)

    def test_cost_function_thrust_deviation(self):
        controls = np.array([[0, 0, 0, 12], [0, 0, 0, 12]])
        self.assertAlmostEqual(cost_function_thrust(np.zeros((3, 6)), controls, 2, 0.1), 0.8, places=6)

    def test_cost_function_angular_zero(self):
        controls = np.array([[0, 0, 0, 10], [0, 0, 0, 10]])
        self.assertAlmostEqual(cost_function_angular(np.zeros((3, 6)), controls, 2, 1.0), 0.0, places=6)

    def test_cost_function_smoothness_constant(self):
        controls = np.array([[0.1, 0.1, 0.1, 10], [0.1, 0.1, 0.1, 10], [0.1, 0.1, 0.1, 10]])
        self.assertAlmostEqual(cost_function_smoothness(np.zeros((4, 6)), controls, 3, 5.0), 0.0, places=6)

    def test_cost_function_gimbal_lock_safe(self):
        states = np.array([[0, 0, 0, 0, 0.5, 0], [0, 0, 0, 0, 0.6, 0], [0, 0, 0, 0, 0.7, 0]])
        self.assertAlmostEqual(cost_function_gimbal_lock(states, np.zeros((2, 4)), 2), 0.0, places=6)

# run tests
suite = unittest.TestSuite()
for test_class in [TestTrajOptHelpers, TestCostFunctions]:
    for method in dir(test_class):
        if method.startswith('test_'):
            suite.addTest(test_class(method))
unittest.TextTestRunner(verbosity=2).run(suite)

test_angle_diff_small (__main__.TestTrajOptHelpers.test_angle_diff_small) ... ok
test_angle_diff_wrapping (__main__.TestTrajOptHelpers.test_angle_diff_wrapping) ... ok
test_pack_unpack_inverse (__main__.TestTrajOptHelpers.test_pack_unpack_inverse) ... ok
test_cost_function_angular_zero (__main__.TestCostFunctions.test_cost_function_angular_zero) ... ok
test_cost_function_gimbal_lock_safe (__main__.TestCostFunctions.test_cost_function_gimbal_lock_safe) ... ok
test_cost_function_smoothness_constant (__main__.TestCostFunctions.test_cost_function_smoothness_constant) ... ok
test_cost_function_thrust_deviation (__main__.TestCostFunctions.test_cost_function_thrust_deviation) ... ok
test_cost_function_thrust_hover (__main__.TestCostFunctions.test_cost_function_thrust_hover) ... ok

----------------------------------------------------------------------
Ran 8 tests in 0.019s

OK


<unittest.runner.TextTestResult run=8 errors=0 failures=0>

## Defining Constraints

Cost alone isn't enough — without constraints, the optimizer might find "solutions" that teleport between waypoints or skip hoops entirely. Constraints make physically impossible trajectories mathematically impossible.

**Equality constraints** ($c(z) = 0$): dynamics, start/goal positions, hoop passages

**Inequality constraints** ($c(z) \leq 0$): collision avoidance with safety margins

For $N$ time steps and $H$ hoops, we have $6N$ dynamics constraints + $9 + 3H$ boundary constraints.

In [30]:
def dynamics_constraints_robust(z: np.ndarray, N: int, dt: float) -> np.ndarray:
    """Enforce physics: position updates via terminal velocity, attitude via angular controls."""
    violations = []
    mass, g = 1.0, 10.0
    states, controls = unpack_decision_vars(z, N)

    for k in range(N):
        px, py, pz, yaw, pitch, roll = states[k, :]
        dyaw, dpitch, droll, thrust = controls[k, :]
        px_next, py_next, pz_next = states[k+1, 0:3]
        yaw_next, pitch_next, roll_next = states[k+1, 3:6]

        # predicted attitude (simple integration)
        yaw_pred, pitch_pred, roll_pred = yaw + dyaw, pitch + dpitch, roll + droll

        # predicted position (terminal velocity model)
        attitude = helpers.rotation_matrix_from_euler(yaw, pitch, roll)
        attitude_rot3 = gtsam.Rot3(attitude)
        force = compute_force_with_gravity(attitude_rot3, thrust, mass)
        v_terminal = compute_terminal_velocity(force)
        pos_pred = np.array([px, py, pz]) + v_terminal * dt

        violations.extend([
            px_next - pos_pred[0], py_next - pos_pred[1], pz_next - pos_pred[2],
            angle_diff(yaw_next, yaw_pred), angle_diff(pitch_next, pitch_pred), angle_diff(roll_next, roll_pred)
        ])

    return np.array(violations)


def boundary_constraints_robust(z: np.ndarray, N: int, start_pose: gtsam.Pose3,
                                goal_position: np.ndarray, hoops: List[gtsam.Pose3]) -> np.ndarray:
    """Enforce start pose (6), goal position (3), and hoop passages (3 per hoop)."""
    violations = []
    states, controls = unpack_decision_vars(z, N)

    # start constraints
    start_pos = start_pose.translation()
    start_rot = start_pose.rotation().matrix()
    yaw_s, pitch_s, roll_s = helpers.euler_from_rotation_matrix_safe(start_rot)
    violations.extend([
        states[0, 0] - start_pos[0], states[0, 1] - start_pos[1], states[0, 2] - start_pos[2],
        angle_diff(states[0, 3], yaw_s), angle_diff(states[0, 4], pitch_s), angle_diff(states[0, 5], roll_s)
    ])

    # goal position constraints
    violations.extend([states[N, 0] - goal_position[0], states[N, 1] - goal_position[1], states[N, 2] - goal_position[2]])

    # hoop constraints
    if len(hoops) > 0:
        positions = states[:, 0:3]
        hoop_positions = [np.array(hoop.translation()) for hoop in hoops]
        hoop_indices = helpers.find_hoop_indices_robust(positions, hoop_positions, N)
        for h_idx, hoop in enumerate(hoops):
            k = hoop_indices[h_idx]
            hoop_pos = hoop.translation()
            violations.extend([states[k, 0] - hoop_pos[0], states[k, 1] - hoop_pos[1], states[k, 2] - hoop_pos[2]])

    return np.array(violations)


def collision_constraints_optimized(z: np.ndarray, N: int, obstacles: List, subsample: int = 2) -> np.ndarray:
    """Inequality constraints for obstacle avoidance. Positive = collision, negative = safe."""
    states, _ = unpack_decision_vars(z, N)
    violations = []
    safety_margin = 0.5

    for k in range(0, N + 1, subsample):
        px, py, pz = states[k, 0:3]
        for obs in obstacles:
            if isinstance(obs, helpers.SphereObstacle):
                dist = np.linalg.norm(np.array([px, py, pz]) - np.array(obs.center))
                violations.append(obs.radius + safety_margin - dist)

    return np.array(violations)

Testing constraints to verify correct dimensions and behavior.

In [31]:
class TestConstraints(unittest.TestCase):
    def test_dynamics_constraints_hover(self):
        N, dt = 2, 0.1
        states = np.array([[5, 5, 5, 0, 0, 0], [5, 5, 5, 0, 0, 0], [5, 5, 5, 0, 0, 0]])
        controls = np.array([[0, 0, 0, 10], [0, 0, 0, 10]])
        z = pack_decision_vars(states, controls, N)
        violations = dynamics_constraints_robust(z, N, dt)
        self.assertEqual(violations.shape[0], 6 * N)
        self.assertTrue(np.max(np.abs(violations)) < 1.0)

    def test_boundary_constraints_dimensions(self):
        N = 5
        start_pose = gtsam.Pose3(gtsam.Rot3(), gtsam.Point3(0, 0, 0))
        goal_position = np.array([5, 5, 5])
        states, controls = np.random.randn(N+1, 6), np.zeros((N, 4))
        z = pack_decision_vars(states, controls, N)

        self.assertEqual(boundary_constraints_robust(z, N, start_pose, goal_position, []).shape[0], 9)
        hoop1 = gtsam.Pose3(gtsam.Rot3(), gtsam.Point3(2, 2, 2))
        self.assertEqual(boundary_constraints_robust(z, N, start_pose, goal_position, [hoop1]).shape[0], 12)

suite = unittest.TestSuite()
for test in ['test_dynamics_constraints_hover', 'test_boundary_constraints_dimensions']:
    suite.addTest(TestConstraints(test))
unittest.TextTestRunner(verbosity=2).run(suite)

test_dynamics_constraints_hover (__main__.TestConstraints.test_dynamics_constraints_hover) ... ok
test_boundary_constraints_dimensions (__main__.TestConstraints.test_boundary_constraints_dimensions) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.005s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

## Running the Optimizer

The optimizer uses SLSQP to minimize cost while satisfying all constraints. We initialize from the RRT path (resampled to N+1 knot points) to give a feasible starting point.

In [32]:
def initialize_from_rrt_robust(rrt_path: List[gtsam.Pose3], N: int, dt: float, start_pose: gtsam.Pose3) -> np.ndarray:
    """Resample RRT path to N+1 knots and compute initial controls."""
    path_length = len(rrt_path)
    states_init = np.zeros((N + 1, 6))

    for i in range(N + 1):
        idx_float = i * (path_length - 1) / N
        idx_low, idx_high = int(np.floor(idx_float)), min(int(np.ceil(idx_float)), path_length - 1)
        alpha = idx_float - idx_low

        pos_low = rrt_path[idx_low].translation()
        pos = pos_low + alpha * (rrt_path[idx_high].translation() - pos_low) if idx_high > idx_low else pos_low

        R_low = rrt_path[idx_low].rotation().matrix()
        yaw, pitch, roll = helpers.euler_from_rotation_matrix_safe(R_low)
        if idx_high > idx_low:
            R_high = rrt_path[idx_high].rotation().matrix()
            yaw_h, pitch_h, roll_h = helpers.euler_from_rotation_matrix_safe(R_high)
            yaw += alpha * angle_diff(yaw_h, yaw)
            pitch += alpha * angle_diff(pitch_h, pitch)
            roll += alpha * angle_diff(roll_h, roll)

        states_init[i, :] = [pos[0], pos[1], pos[2], yaw, pitch, roll]

    controls_init = np.zeros((N, 4))
    for k in range(N):
        controls_init[k, :3] = [angle_diff(states_init[k+1, 3], states_init[k, 3]),
                                angle_diff(states_init[k+1, 4], states_init[k, 4]),
                                angle_diff(states_init[k+1, 5], states_init[k, 5])]
        controls_init[k, 3] = 10.0

    deg_to_rad = np.pi / 180
    controls_init[:, 0:3] = np.clip(controls_init[:, 0:3], -10 * deg_to_rad, 10 * deg_to_rad)
    controls_init[:, 3] = np.clip(controls_init[:, 3], 5, 20)

    return pack_decision_vars(states_init, controls_init, N)

## Simple Path Optimization Demo

Testing optimization on a simple point-to-point path without hoops.

In [33]:
start_simple = gtsam.Pose3(gtsam.Rot3(), gtsam.Point3(2, 2, 5))
goal_simple = gtsam.Pose3(gtsam.Rot3(), gtsam.Point3(8, 8, 5))

print("Running RRT...")
rrt_simple, parents_simple = run_rrt(start_simple, goal_simple, generate_random_pose, steer,
                                      helpers.distance_between_poses, find_nearest_pose, threshold=2.0)
path_simple = get_rrt_path(rrt_simple, parents_simple)

print(f"Optimizing {len(path_simple)} waypoints...")
optimized_simple, success, info = helpers.optimize_trajectory(
    rrt_path=path_simple, start_pose=start_simple, goal_position=goal_simple.translation(),
    hoops=[], obstacles=[], N=20, dt=0.1,
    weights={'thrust': 0.1, 'angular': 1.0, 'smoothness': 5.0},
    initialize_from_rrt_robust=initialize_from_rrt_robust,
    dynamics_constraints_robust=dynamics_constraints_robust,
    boundary_constraints_robust=boundary_constraints_robust,
    collision_constraints_optimized=collision_constraints_optimized,
    cost_function_tuned=cost_function_tuned,
    unpack_decision_vars=unpack_decision_vars
)

print(f"{'SUCCESS' if success else 'FAILED'} | Cost: {info.get('cost', 'N/A')}")
helpers.visualize_rrt_vs_optimized_comparison(path_simple, optimized_simple, start_simple, goal_simple).show()

Running RRT...
Optimizing 11 waypoints...

TRAJECTORY OPTIMIZATION
Knot points: 20, Time step: 0.1s, Duration: 2.0s
Decision variables: 206

Initializing from RRT...
Setting up constraints...

Optimizing with SLSQP...



/usr/local/lib/python3.11/dist-packages/scipy/optimize/_slsqp_py.py:435: RuntimeWarning:

Values in x were outside bounds during a minimize step, clipping to bounds



Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.0017061650245196484
            Iterations: 173
            Function evaluations: 35878
            Gradient evaluations: 173

✅ OPTIMIZATION SUCCEEDED
Status: Optimization terminated successfully
Iterations: 173
Final cost: 0.0017
Optimization time: 60.56s
Max constraint violation: 0.000002

SUCCESS | Cost: 0.0017061650245196484


## Racing Path Optimization

Optimizing the full racing trajectory through all hoops.

In [34]:
hoops_race = helpers.get_hoops()
targets_race = helpers.get_targets()
start_race = gtsam.Pose3(gtsam.Rot3(), gtsam.Point3(1, 3, 8))

print("Running RRT through hoops...")
rrt_race_path = drone_racing_rrt(start_race, targets_race)

print("Optimizing racing trajectory...")
optimized_race, success_race, info_race = helpers.optimize_racing_path_sequential(
    rrt_path=rrt_race_path, start_pose=start_race, hoops=hoops_race, obstacles=[], N=25, dt=0.1,
    weights={'thrust': 0.05, 'angular': 0.5, 'smoothness': 5.0},
    initialize_from_rrt_robust=initialize_from_rrt_robust,
    dynamics_constraints_robust=dynamics_constraints_robust,
    boundary_constraints_robust=boundary_constraints_robust,
    collision_constraints_optimized=collision_constraints_optimized,
    cost_function_tuned=cost_function_tuned,
    unpack_decision_vars=unpack_decision_vars
)

print(f"Racing optimization: {'SUCCESS' if success_race else 'FAILED'}")
helpers.drone_racing_path_comparison(hoops_race, start_race, rrt_race_path, optimized_race).show()

Running RRT through hoops...
Planning to hoop 1/4...
Planning to hoop 2/4...
Planning to hoop 3/4...
Planning to hoop 4/4...
Optimizing racing trajectory...

SEQUENTIAL RACING PATH OPTIMIZATION
Hoops: 4, Knot points per segment: 25

Splitting path into segments:
  Hoop 1 at RRT waypoint 10
  Hoop 2 at RRT waypoint 14
  Hoop 3 at RRT waypoint 29
  Hoop 4 at RRT waypoint 44

------------------------------------------------------------
SEGMENT 1/4 → Hoop 1
------------------------------------------------------------


TRAJECTORY OPTIMIZATION
Knot points: 25, Time step: 0.1s, Duration: 2.5s
Decision variables: 256

Initializing from RRT...
Setting up constraints...

Optimizing with SLSQP...

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.0005440426387930135
            Iterations: 244
            Function evaluations: 62791
            Gradient evaluations: 244

✅ OPTIMIZATION SUCCEEDED
Status: Optimization terminated successfully
Iterations: 24

/usr/local/lib/python3.11/dist-packages/scipy/optimize/_slsqp_py.py:439: RuntimeWarning:

Values in x were outside bounds during a minimize step, clipping to bounds

/usr/local/lib/python3.11/dist-packages/scipy/optimize/_slsqp_py.py:493: RuntimeWarning:

Values in x were outside bounds during a minimize step, clipping to bounds



Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.009688393324938158
            Iterations: 276
            Function evaluations: 71041
            Gradient evaluations: 276

✅ OPTIMIZATION SUCCEEDED
Status: Optimization terminated successfully
Iterations: 276
Final cost: 0.0097
Optimization time: 148.66s
Max constraint violation: 0.000002

✅ Segment 2 succeeded

------------------------------------------------------------
SEGMENT 3/4 → Hoop 3
------------------------------------------------------------


TRAJECTORY OPTIMIZATION
Knot points: 25, Time step: 0.1s, Duration: 2.5s
Decision variables: 256

Initializing from RRT...
Setting up constraints...

Optimizing with SLSQP...

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.38560587546980485
            Iterations: 297
            Function evaluations: 76427
            Gradient evaluations: 297

✅ OPTIMIZATION SUCCEEDED
Status: Optimization termina

# Final Challenge — Racing with Obstacles

Everything comes together: RRT planning + trajectory optimization + obstacle avoidance. The optimizer must satisfy dynamics, boundary, hoop, AND collision constraints simultaneously.

In [35]:
import time

hoops_final = helpers.get_hoops()
targets_final = helpers.get_targets()
start_final = gtsam.Pose3(gtsam.Rot3(), gtsam.Point3(1, 3, 8))
obstacles_easy = helpers.get_obstacles_easy()

print(f"Racing: {len(hoops_final)} hoops, {len(obstacles_easy)} obstacles")

# RRT with obstacles
t0 = time.time()
rrt_final_path = drone_racing_rrt_with_obstacles(start_final, targets_final, obstacles_easy)
print(f"RRT: {len(rrt_final_path)} waypoints in {time.time()-t0:.2f}s")

# Optimize
t0 = time.time()
optimized_final, success_final, _ = helpers.optimize_racing_path_sequential(
    rrt_path=rrt_final_path, start_pose=start_final, hoops=hoops_final, obstacles=obstacles_easy,
    N=25, dt=0.1, weights={'thrust': 0.05, 'angular': 0.5, 'smoothness': 5.0},
    initialize_from_rrt_robust=initialize_from_rrt_robust,
    dynamics_constraints_robust=dynamics_constraints_robust,
    boundary_constraints_robust=boundary_constraints_robust,
    collision_constraints_optimized=collision_constraints_optimized,
    cost_function_tuned=cost_function_tuned,
    unpack_decision_vars=unpack_decision_vars
)
print(f"Optimization: {'SUCCESS' if success_final else 'FAILED'} in {time.time()-t0:.2f}s")

has_collision, _ = helpers.check_path_collision(optimized_final if success_final else rrt_final_path, obstacles_easy)
print(f"Collision-free: {not has_collision}")

helpers.drone_racing_path_with_obstacles(hoops_final, start_final, optimized_final if success_final else rrt_final_path, obstacles_easy)

Racing: 4 hoops, 2 obstacles
Planning to hoop 1/4...
Goal reached in 193 iterations
Planning to hoop 2/4...
Goal reached in 34 iterations
Planning to hoop 3/4...
Goal reached in 50 iterations
Planning to hoop 4/4...
Goal reached in 120 iterations
RRT: 62 waypoints in 2.31s

SEQUENTIAL RACING PATH OPTIMIZATION
Hoops: 4, Knot points per segment: 25

Splitting path into segments:
  Hoop 1 at RRT waypoint 14
  Hoop 2 at RRT waypoint 26
  Hoop 3 at RRT waypoint 47
  Hoop 4 at RRT waypoint 60

------------------------------------------------------------
SEGMENT 1/4 → Hoop 1
------------------------------------------------------------


TRAJECTORY OPTIMIZATION
Knot points: 25, Time step: 0.1s, Duration: 2.5s
Decision variables: 256

Initializing from RRT...
Setting up constraints...
  Added collision avoidance for 2 obstacles

Optimizing with SLSQP...

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.0007006743576075546
            Iterations: 317
  

/usr/local/lib/python3.11/dist-packages/scipy/optimize/_slsqp_py.py:499: RuntimeWarning:

Values in x were outside bounds during a minimize step, clipping to bounds



Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.006844726612863444
            Iterations: 383
            Function evaluations: 98529
            Gradient evaluations: 383

✅ OPTIMIZATION SUCCEEDED
Status: Optimization terminated successfully
Iterations: 383
Final cost: 0.0068
Optimization time: 203.67s
Max constraint violation: 0.000004

✅ Segment 2 succeeded

------------------------------------------------------------
SEGMENT 3/4 → Hoop 3
------------------------------------------------------------


TRAJECTORY OPTIMIZATION
Knot points: 25, Time step: 0.1s, Duration: 2.5s
Decision variables: 256

Initializing from RRT...
Setting up constraints...
  Added collision avoidance for 2 obstacles

Optimizing with SLSQP...

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.44729813788322503
            Iterations: 251
            Function evaluations: 64594
            Gradient evaluations: 251

✅ OPTIMIZ

# Hard Obstacle Course & Custom Planners

The easy course was a warmup. The hard course has **8 obstacles** (spheres AND boxes) with narrow gaps. Your collision constraints must handle both obstacle types, and RRT may need multiple attempts.

**Challenge:**
- Extend `collision_constraints_optimized` to handle box obstacles
- Implement a custom planner (RRT*, Informed RRT*, etc.)
- Navigate the hard course collision-free

In [36]:
# extended collision constraints handling both spheres and boxes
def collision_constraints_extended(z: np.ndarray, N: int, obstacles: List, subsample: int = 2) -> np.ndarray:
    """Collision constraints for both sphere and box obstacles."""
    states, _ = unpack_decision_vars(z, N)
    violations = []
    safety_margin = 0.5

    for k in range(0, N + 1, subsample):
        pos = states[k, 0:3]

        for obs in obstacles:
            if isinstance(obs, helpers.SphereObstacle):
                dist = np.linalg.norm(pos - np.array(obs.center))
                violations.append(obs.radius + safety_margin - dist)

            elif isinstance(obs, helpers.BoxObstacle):
                # for box: positive violation if inside, negative if outside
                min_corner = np.array(obs.min_corner) - safety_margin
                max_corner = np.array(obs.max_corner) + safety_margin

                # distance to nearest face (negative = inside)
                dist_to_min = pos - min_corner
                dist_to_max = max_corner - pos

                # if all distances positive, we're inside the expanded box
                min_dist = min(np.min(dist_to_min), np.min(dist_to_max))
                violations.append(-min_dist)  # positive when inside

    return np.array(violations)

## RRT* Implementation

RRT* improves on basic RRT by:
1. **Best parent selection:** Connect to lowest-cost neighbor, not just nearest
2. **Rewiring:** Optimize tree structure when better paths are found
3. **Cost tracking:** Minimize total path length

This produces asymptotically optimal paths.

In [37]:
def rrt_star_enhanced(start, target, obstacles, max_iterations=6000, threshold=2.5, search_radius=3.5):
    """RRT* with rewiring for asymptotically optimal paths."""
    rrt, parents, costs = [start], [-1], [0.0]

    for i in range(max_iterations):
        # sample with goal bias
        rand_node = target if rng.random() < 0.2 else generate_random_pose(target)
        nearest, nearest_idx = find_nearest_pose(rrt, rand_node)
        new_node = steer(nearest, rand_node)

        # collision check
        p_nearest = nearest.translation() if isinstance(nearest, gtsam.Pose3) else nearest
        p_new = new_node.translation() if isinstance(new_node, gtsam.Pose3) else new_node
        if helpers.check_segment_collision(p_nearest, p_new, obstacles):
            continue

        # find best parent in neighborhood
        best_parent_idx, best_cost = nearest_idx, costs[nearest_idx] + helpers.distance_between_poses(nearest, new_node)

        for j, node in enumerate(rrt):
            dist = helpers.distance_between_poses(node, new_node)
            if 0.01 < dist < search_radius:
                potential_cost = costs[j] + dist
                if potential_cost < best_cost:
                    p_node = node.translation() if isinstance(node, gtsam.Pose3) else node
                    if not helpers.check_segment_collision(p_node, p_new, obstacles):
                        best_cost, best_parent_idx = potential_cost, j

        # add node
        rrt.append(new_node)
        parents.append(best_parent_idx)
        costs.append(best_cost)
        new_idx = len(rrt) - 1

        # rewire nearby nodes
        for j in range(len(rrt) - 1):
            node = rrt[j]
            dist = helpers.distance_between_poses(new_node, node)
            if 0.01 < dist < search_radius:
                cost_through_new = best_cost + dist
                if cost_through_new < costs[j]:
                    p_node = node.translation() if isinstance(node, gtsam.Pose3) else node
                    if not helpers.check_segment_collision(p_new, p_node, obstacles):
                        parents[j], costs[j] = new_idx, cost_through_new

        if helpers.distance_between_poses(new_node, target) < threshold:
            print(f"Goal reached in {i+1} iterations (cost: {best_cost:.2f})")
            return rrt, parents

    print("Max iterations reached")
    return rrt, parents


def drone_racing_rrt_star(start: gtsam.Pose3, targets: List[gtsam.Pose3], obstacles: List = None) -> List[gtsam.Pose3]:
    """Sequential racing using RRT* for each segment."""
    if obstacles is None:
        obstacles = []

    drone_path = []
    current_start = start

    for i, target in enumerate(targets):
        print(f"Segment {i+1}/{len(targets)}...")
        rrt, parents = rrt_star_enhanced(current_start, target, obstacles)
        segment_path = get_rrt_path(rrt, parents)
        helpers.pass_through_the_hoop(target, segment_path)
        drone_path.extend(segment_path if i == 0 else segment_path[1:])
        current_start = segment_path[-1]

    return drone_path

## Testing on Hard Course

Running RRT* on the hard obstacle course with 8 obstacles.

In [38]:
import time

start_hard = gtsam.Pose3(gtsam.Rot3(), gtsam.Point3(1, 3, 8))
targets_hard = helpers.get_targets()
obstacles_hard = helpers.get_obstacles_hard()

print(f"Hard course: {len(targets_hard)} hoops, {len(obstacles_hard)} obstacles")

# visualize the challenge
helpers.drone_racing_path_with_obstacles(helpers.get_hoops(), start_hard, [], obstacles=obstacles_hard)

Hard course: 4 hoops, 8 obstacles


In [39]:
# run RRT* on hard course
print("\nRunning RRT*...")
t_start = time.time()
path_rrt_star = drone_racing_rrt_star(start_hard, targets_hard, obstacles_hard)
t_elapsed = time.time() - t_start

print(f"\nRRT* completed in {t_elapsed:.2f}s")
print(f"Path: {len(path_rrt_star)} waypoints")

has_collision, _ = helpers.check_path_collision(path_rrt_star, obstacles_hard)
print(f"Collision-free: {not has_collision}")

helpers.drone_racing_path_with_obstacles(helpers.get_hoops(), start_hard, path_rrt_star, obstacles_hard)


Running RRT*...
Segment 1/4...
Goal reached in 5 iterations (cost: 5.56)
Segment 2/4...
Goal reached in 11 iterations (cost: 12.15)
Segment 3/4...
Goal reached in 67 iterations (cost: 25.16)
Segment 4/4...
Goal reached in 273 iterations (cost: 7.32)

RRT* completed in 3.15s
Path: 33 waypoints
Collision-free: False


## Optimization on Hard Course

If RRT* succeeds, we optimize the path for smoothness while maintaining collision avoidance.

In [40]:
if not has_collision and len(path_rrt_star) > 0:
    print("Optimizing hard course trajectory...")

    t_opt = time.time()
    optimized_hard, success_hard, info_hard = helpers.optimize_racing_path_sequential(
        rrt_path=path_rrt_star, start_pose=start_hard, hoops=helpers.get_hoops(),
        obstacles=obstacles_hard, N=30, dt=0.1,
        weights={'thrust': 0.03, 'angular': 0.3, 'smoothness': 8.0},
        initialize_from_rrt_robust=initialize_from_rrt_robust,
        dynamics_constraints_robust=dynamics_constraints_robust,
        boundary_constraints_robust=boundary_constraints_robust,
        collision_constraints_optimized=collision_constraints_extended,  # use extended version
        cost_function_tuned=cost_function_tuned,
        unpack_decision_vars=unpack_decision_vars
    )

    print(f"Optimization: {'SUCCESS' if success_hard else 'FAILED'} in {time.time()-t_opt:.2f}s")

    final_path = optimized_hard if success_hard else path_rrt_star
    has_collision_final, _ = helpers.check_path_collision(final_path, obstacles_hard)

    print(f"\nFinal Results:")
    print(f"  RRT* path: {len(path_rrt_star)} waypoints")
    print(f"  Optimized path: {len(final_path)} waypoints")
    print(f"  Collision-free: {not has_collision_final}")

    helpers.drone_racing_path_with_obstacles(helpers.get_hoops(), start_hard, final_path, obstacles_hard)
else:
    print("RRT* failed - try adjusting parameters or retry")

RRT* failed - try adjusting parameters or retry


## Performance Analysis

Quantitative comparison of planning algorithms and optimization results across different scenarios.

In [41]:
import pandas as pd

def benchmark_planners(start, targets, obstacles, num_trials=5):
    """Benchmark RRT vs RRT* on the same scenario."""
    results = []

    for trial in range(num_trials):
        # RRT benchmark
        t0 = time.time()
        try:
            path_rrt = drone_racing_rrt_with_obstacles(start, targets, obstacles)
            rrt_time = time.time() - t0
            rrt_length = compute_path_length(path_rrt)
            rrt_collision, _ = helpers.check_path_collision(path_rrt, obstacles)
            results.append({
                'Algorithm': 'RRT',
                'Trial': trial + 1,
                'Time (s)': rrt_time,
                'Path Length (m)': rrt_length,
                'Waypoints': len(path_rrt),
                'Collision-Free': not rrt_collision
            })
        except Exception as e:
            results.append({'Algorithm': 'RRT', 'Trial': trial + 1, 'Time (s)': None,
                          'Path Length (m)': None, 'Waypoints': None, 'Collision-Free': False})

        # RRT* benchmark
        t0 = time.time()
        try:
            path_rrt_star = drone_racing_rrt_star(start, targets, obstacles)
            rrt_star_time = time.time() - t0
            rrt_star_length = compute_path_length(path_rrt_star)
            rrt_star_collision, _ = helpers.check_path_collision(path_rrt_star, obstacles)
            results.append({
                'Algorithm': 'RRT*',
                'Trial': trial + 1,
                'Time (s)': rrt_star_time,
                'Path Length (m)': rrt_star_length,
                'Waypoints': len(path_rrt_star),
                'Collision-Free': not rrt_star_collision
            })
        except Exception as e:
            results.append({'Algorithm': 'RRT*', 'Trial': trial + 1, 'Time (s)': None,
                          'Path Length (m)': None, 'Waypoints': None, 'Collision-Free': False})

    return pd.DataFrame(results)

# run benchmark on easy course
print("Running benchmark (this may take a few minutes)...")
start_bench = gtsam.Pose3(gtsam.Rot3(), gtsam.Point3(1, 3, 8))
targets_bench = helpers.get_targets()
obstacles_bench = helpers.get_obstacles_easy()

benchmark_df = benchmark_planners(start_bench, targets_bench, obstacles_bench, num_trials=3)
print("\nBenchmark Results:")
print(benchmark_df.to_string(index=False))

# summary statistics
print("\n" + "="*60)
print("Summary Statistics:")
print("="*60)
summary = benchmark_df.groupby('Algorithm').agg({
    'Time (s)': ['mean', 'std'],
    'Path Length (m)': ['mean', 'std'],
    'Collision-Free': 'sum'
}).round(3)
print(summary)

Running benchmark (this may take a few minutes)...
Planning to hoop 1/4...
Goal reached in 7 iterations
Planning to hoop 2/4...
Goal reached in 15 iterations
Planning to hoop 3/4...
Goal reached in 46 iterations
Planning to hoop 4/4...
Goal reached in 984 iterations
Segment 1/4...
Goal reached in 12 iterations (cost: 7.80)
Segment 2/4...
Goal reached in 26 iterations (cost: 12.45)
Segment 3/4...
Goal reached in 63 iterations (cost: 21.44)
Segment 4/4...
Goal reached in 158 iterations (cost: 14.67)
Planning to hoop 1/4...
Goal reached in 13 iterations
Planning to hoop 2/4...
Planning to hoop 3/4...
Goal reached in 126 iterations
Planning to hoop 4/4...
Goal reached in 175 iterations
Segment 1/4...
Goal reached in 5 iterations (cost: 6.48)
Segment 2/4...
Goal reached in 23 iterations (cost: 14.00)
Segment 3/4...
Goal reached in 28 iterations (cost: 17.49)
Segment 4/4...
Goal reached in 34 iterations (cost: 11.87)
Planning to hoop 1/4...
Goal reached in 8 iterations
Planning to hoop 2/4..

In [42]:
def create_comprehensive_analysis_figure(rrt_path, optimized_path, obstacles, info=None):
    """Create a multi-panel analysis figure."""
    from plotly.subplots import make_subplots

    fig = make_subplots(
        rows=2, cols=2,
        specs=[[{'type': 'scatter3d'}, {'type': 'xy'}],
               [{'type': 'xy'}, {'type': 'xy'}]],
        subplot_titles=('3D Trajectory Comparison', 'Path Length Comparison',
                       'Velocity Profile', 'Control Inputs')
    )

    # 3D trajectory
    rrt_pos = np.array([p.translation() for p in rrt_path])
    opt_pos = np.array([p.translation() for p in optimized_path])

    fig.add_trace(go.Scatter3d(x=rrt_pos[:,0], y=rrt_pos[:,1], z=rrt_pos[:,2],
                               mode='lines', name='RRT', line=dict(color='cyan', width=3)), row=1, col=1)
    fig.add_trace(go.Scatter3d(x=opt_pos[:,0], y=opt_pos[:,1], z=opt_pos[:,2],
                               mode='lines', name='Optimized', line=dict(color='magenta', width=3)), row=1, col=1)

    # path length bar chart
    lengths = [compute_path_length(rrt_path), compute_path_length(optimized_path)]
    fig.add_trace(go.Bar(x=['RRT', 'Optimized'], y=lengths, marker_color=['cyan', 'magenta']), row=1, col=2)

    # velocity profile (if optimization data available)
    if info and 'states' in info:
        dt = 0.1
        velocities = np.diff(info['states'][:, :3], axis=0) / dt
        speeds = np.linalg.norm(velocities, axis=1)
        fig.add_trace(go.Scatter(y=speeds, mode='lines', name='Speed', line=dict(color='green')), row=2, col=1)

    # control inputs
    if info and 'controls' in info:
        controls = info['controls']
        fig.add_trace(go.Scatter(y=controls[:, 3], mode='lines', name='Thrust', line=dict(color='red')), row=2, col=2)

    fig.update_layout(height=800, title_text="Comprehensive Trajectory Analysis")
    return fig

# example usage (uncomment when paths are available)
# fig = create_comprehensive_analysis_figure(rrt_race_path, optimized_race, obstacles_easy, info_race)
# fig.show()

## Parameter Sensitivity Analysis

Understanding how different parameters affect optimization performance and trajectory quality.

In [43]:
def sensitivity_analysis(rrt_path, start_pose, hoops, obstacles):
    """Analyze how cost weights affect optimization results."""
    weight_configs = [
        {'thrust': 0.01, 'angular': 0.5, 'smoothness': 10.0, 'name': 'Smooth Priority'},
        {'thrust': 0.1, 'angular': 1.0, 'smoothness': 5.0, 'name': 'Balanced'},
        {'thrust': 0.2, 'angular': 0.3, 'smoothness': 2.0, 'name': 'Aggressive'},
    ]

    results = []
    for config in weight_configs:
        weights = {k: v for k, v in config.items() if k != 'name'}

        opt_path, success, info = helpers.optimize_racing_path_sequential(
            rrt_path=rrt_path, start_pose=start_pose, hoops=hoops, obstacles=obstacles,
            N=20, dt=0.1, weights=weights,
            initialize_from_rrt_robust=initialize_from_rrt_robust,
            dynamics_constraints_robust=dynamics_constraints_robust,
            boundary_constraints_robust=boundary_constraints_robust,
            collision_constraints_optimized=collision_constraints_optimized,
            cost_function_tuned=cost_function_tuned,
            unpack_decision_vars=unpack_decision_vars
        )

        results.append({
            'Config': config['name'],
            'Success': success,
            'Final Cost': info.get('cost', 'N/A'),
            'Iterations': info.get('iterations', 'N/A'),
            'Path Length': compute_path_length(opt_path) if success else 'N/A'
        })

    return pd.DataFrame(results)

# uncomment to run sensitivity analysis
# sensitivity_df = sensitivity_analysis(rrt_race_path, start_race, hoops_race, [])
# print(sensitivity_df)

## Failure Modes & Debugging

Common issues encountered during development and their solutions.

| Issue | Symptom | Solution |
|-------|---------|----------|
| RRT timeout | No path found after max iterations | Increase iterations, adjust goal bias, check obstacle configuration |
| Optimization divergence | Cost increases or NaN values | Reduce weights, improve initialization, check constraint scaling |
| Collision after optimization | Path intersects obstacles | Increase collision subsample density, adjust safety margins |
| Gimbal lock | Erratic attitude near ±90° pitch | Add gimbal lock penalty to cost function |
| Constraint violation | Non-zero constraint residuals | Check dynamics equations, verify angle wrapping |

## Future Work

Potential extensions to enhance this system:

**Algorithm Improvements:**
- Implement Informed RRT* with heuristic-guided sampling
- Add kinodynamic RRT for direct dynamics integration
- Explore MPPI (Model Predictive Path Integral) for real-time replanning

**Dynamics Enhancements:**
- Full 12-DOF quadrotor dynamics with motor models
- Aerodynamic effects (wind, ground effect)
- Battery and weight constraints

**Real-World Deployment:**
- ROS2 integration for hardware deployment
- Sensor fusion with VIO (Visual-Inertial Odometry)
- Model Predictive Control (MPC) for trajectory tracking

**Learning-Based Extensions:**
- Neural network collision checking for speedup
- Reinforcement learning for adaptive racing strategies
- Imitation learning from expert racing data

## References

1. LaValle, S. M. (1998). *Rapidly-exploring random trees: A new tool for path planning*. Technical Report TR 98-11, Computer Science Department, Iowa State University.

2. Karaman, S., & Frazzoli, E. (2011). *Sampling-based algorithms for optimal motion planning*. The International Journal of Robotics Research, 30(7), 846-894.

3. Tedrake, R. (2023). *Underactuated Robotics: Algorithms for Walking, Running, Swimming, Flying, and Manipulation*. MIT. [underactuated.mit.edu](http://underactuated.mit.edu)

4. GTSAM Documentation. Georgia Tech. [gtsam.org](https://gtsam.org)

5. Mellinger, D., & Kumar, V. (2011). *Minimum snap trajectory generation and control for quadrotors*. IEEE International Conference on Robotics and Automation.

---

## Acknowledgments

- Course instructors and TAs for the helper functions and framework
- GTSAM team for the factor graph library
- Open-source robotics community

---

**Author:** Meet Brahmbhatt  
**Date:** November 2024  
**Repository:** [github.com/brahmbhatt-me/drone_racing_project](https://github.com/brahmbhatt-me/drone_racing_project)

In [44]:
def generate_project_summary():
    """Generate a summary of all implemented functions for documentation."""
    functions = {
        'Path Planning': [
            ('generate_random_point', 'Samples random 3D points with goal bias'),
            ('generate_random_pose', 'Samples random 6-DOF poses'),
            ('find_nearest_node', 'Vectorized nearest neighbor search'),
            ('find_nearest_pose', 'Pose-space nearest neighbor with SO(3) distance'),
            ('run_rrt', 'Basic RRT algorithm'),
            ('run_rrt_with_obstacles', 'RRT with collision checking'),
            ('rrt_star_enhanced', 'RRT* with rewiring'),
        ],
        'Drone Dynamics': [
            ('compute_attitude_from_ypr', 'Euler angles to rotation matrix'),
            ('compute_force', 'Thrust to force in navigation frame'),
            ('compute_force_with_gravity', 'Net force including gravity'),
            ('compute_terminal_velocity', 'Force to velocity via drag model'),
            ('steer_naive', 'Simple linear interpolation steering'),
            ('steer_with_terminal_velocity', 'Physics-based steering'),
            ('steer', 'Realistic steering with attitude limits'),
        ],
        'Trajectory Optimization': [
            ('pack_decision_vars', 'States/controls to flat vector'),
            ('unpack_decision_vars', 'Flat vector to states/controls'),
            ('angle_diff', 'Wrapped angular difference'),
            ('cost_function_thrust', 'Thrust deviation penalty'),
            ('cost_function_angular', 'Angular velocity penalty'),
            ('cost_function_smoothness', 'Control jerk penalty'),
            ('cost_function_gimbal_lock', 'Singularity avoidance'),
            ('dynamics_constraints_robust', 'Physics enforcement'),
            ('boundary_constraints_robust', 'Start/goal/hoop constraints'),
            ('collision_constraints_optimized', 'Obstacle avoidance'),
        ],
        'Racing': [
            ('drone_racing_rrt', 'Multi-hoop sequential RRT'),
            ('drone_racing_rrt_with_obstacles', 'Racing with collision avoidance'),
            ('drone_racing_rrt_star', 'Racing with RRT*'),
            ('get_rrt_path', 'Path extraction from tree'),
        ]
    }

    print("="*70)
    print("PROJECT FUNCTION SUMMARY")
    print("="*70)

    total = 0
    for category, funcs in functions.items():
        print(f"\n{category}:")
        print("-" * 40)
        for name, desc in funcs:
            print(f"  • {name:35s} {desc}")
            total += 1

    print(f"\n{'='*70}")
    print(f"Total implemented functions: {total}")
    print("="*70)

generate_project_summary()

PROJECT FUNCTION SUMMARY

Path Planning:
----------------------------------------
  • generate_random_point               Samples random 3D points with goal bias
  • generate_random_pose                Samples random 6-DOF poses
  • find_nearest_node                   Vectorized nearest neighbor search
  • find_nearest_pose                   Pose-space nearest neighbor with SO(3) distance
  • run_rrt                             Basic RRT algorithm
  • run_rrt_with_obstacles              RRT with collision checking
  • rrt_star_enhanced                   RRT* with rewiring

Drone Dynamics:
----------------------------------------
  • compute_attitude_from_ypr           Euler angles to rotation matrix
  • compute_force                       Thrust to force in navigation frame
  • compute_force_with_gravity          Net force including gravity
  • compute_terminal_velocity           Force to velocity via drag model
  • steer_naive                         Simple linear interpolation steeri

## Summary

This project implemented a complete autonomous drone racing pipeline:

1. **3D RRT** for collision-free path planning
2. **Drone dynamics** with terminal velocity model
3. **Physics-based steering** with gravity and drag
4. **Obstacle avoidance** integrated into planning
5. **Trajectory optimization** via direct transcription
6. **RRT*** for asymptotically optimal paths

The same techniques power real autonomous systems in drone racing, self-driving cars, and spacecraft landing.